# Random Forest

One of the models we chose to implement for the purpose of lung cancer classification was Random Forest, an ensemble learning method that constructs a multitude of decision trees at training time.  
This algorithm is suitable for classification prediction problems, handles a lot of features well and lets us easily check the relative importance assigned to each.

We start by importing relevant libraries and dropping useless columns from our CSV.

In [ ]:
import pandas as pd
import numpy as np
from kfold_and_metrics import *

from sklearn.metrics import roc_auc_score
from sklearn.ensemble import RandomForestClassifier

In [ ]:
df = pd.read_csv("final.csv")
df = df.drop(columns=["id"])
df.head()

### Parameter Hypertuning

In order to prevent overfitting, we hypertuned our model on the values of the number of estimators, max depth and max samples per leaf.  
We also tested which of the evaluation criteria suited our data best.

In [ ]:
best_auc = 0
best = {}

for crit in ["gini", "entropy", "log_loss"]:
    for n_est in range(30, 201, 10):
        for m_depth in range(10, 31):
            for m_samples_leaf in range(5, 21):
                params = {'n_estimators': n_est, 'max_depth': m_depth, 'min_samples_leaf': m_samples_leaf, 'criterion': crit}
                print("Current parameter combination:")
                for parameter, value in params.items():
                    print(f"\t{parameter}: {value}")
                print()

                rf_model = RandomForestClassifier(n_estimators=n_est, criterion=crit, max_depth=m_depth, min_samples_leaf=m_samples_leaf)
                score = k_fold_cv(model=rf_model, df=df, metric_funcs=[roc_auc_score])
                avg_auc, std = weighted_avg_and_std(np.array(score["roc_auc_score"]))
                if avg_auc > best_auc:
                    best_auc = avg_auc
                    best = params

After running, we got the following values:

In [ ]:
print("Results of the grid search parameter hypertunning:")
for parameter, value in best.items():
    print(f"\t{parameter}: {value}")